# Notebook to Import and Merge Together Public Data

In this notebook, we will merge together data coming from different datasets. 
We will finally have 6 final datasets for years from 2015 to 2021, since these are the years common to most of our datasets. Note that not all of the initial datasets are used, since some of them are missing too many countries and years, and it would mean to have to deal with too many missing values, potential source of bias.
In our later analysis we can be free to use one of these datasets, depending on our final scope (which disease, which measure).
The 36 datasets are the results of matching different diseases with different measures. We kept only the **rate** as final metric.
In particular:
- disease: **COPD** (Chronic obstructive pulmonary disease) or **asthma** or **tuberculosis**
- measure: **incidence**, **prevalence**

So finally we have: 3 x 2 = 6 datasets

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cfgrib
import xarray as xr
from functools import reduce

In [3]:
# Définir le chemin de base
BASE_PATH = "/mnt/c/Users/ala78/OneDrive - HESSO/BA5/GML/data-gml-health/"

# Définir le dossier de sauvegarde
folder_name = "Refined/9019/"

## Respiratory disease data - 1990 to 2021 (no sex or age)

In [4]:
# Votre code ici:
df_diseases = pd.read_csv(f"{BASE_PATH}health/IHME/IHME-data-allCountries-allYears/IHME-GBD_2023_DATA-8de3a169-1.csv")
df_diseases = df_diseases[["measure", "location", "cause", "metric", "year", "val"]]
df_diseases.columns = ["Measure", "Country Name", "Disease", "Metric", "Year", "Value"]
df_diseases

,Measure,Country Name,Disease,Metric,Year,Value
0,Prevalence,Taiwan,Chronic obstructive pulmonary disease,Rate,1990,1366.915295
1,Prevalence,Taiwan,Chronic obstructive pulmonary disease,Rate,1991,1398.950832
2,Prevalence,Taiwan,Chronic obstructive pulmonary disease,Rate,1992,1432.737779
3,Prevalence,Taiwan,Chronic obstructive pulmonary disease,Rate,1993,1465.397615
4,Prevalence,Taiwan,Chronic obstructive pulmonary disease,Rate,1994,1504.531328
...,...,...,...,...,...,...
41611,Incidence,Niue,Tuberculosis,Rate,2019,39.523161
41612,Incidence,Niue,Tuberculosis,Rate,2020,39.049782
41613,Incidence,Niue,Tuberculosis,Rate,2021,38.504013
41614,Incidence,Niue,Tuberculosis,Rate,2022,37.588972


In [5]:
df_countries = pd.read_csv(f"{BASE_PATH}economic/wikipedia-iso-country-codes.csv")
df_countries = df_countries[['English short name lower case', 'Alpha-3 code']]
df_countries.rename(columns={'English short name lower case': 'Country Name'}, inplace=True)
df_diseases = pd.merge(
        df_diseases, df_countries,
        on=["Country Name"],
        how="left",
        suffixes=("_1", "_2")
    )
df_diseases.rename(columns={'Alpha-3 code': 'Country Code'}, inplace=True)
df_diseases

,Measure,Country Name,Disease,Metric,Year,Value,Country Code
0,Prevalence,Taiwan,Chronic obstructive pulmonary disease,Rate,1990,1366.915295,TWN
1,Prevalence,Taiwan,Chronic obstructive pulmonary disease,Rate,1991,1398.950832,TWN
2,Prevalence,Taiwan,Chronic obstructive pulmonary disease,Rate,1992,1432.737779,TWN
3,Prevalence,Taiwan,Chronic obstructive pulmonary disease,Rate,1993,1465.397615,TWN
4,Prevalence,Taiwan,Chronic obstructive pulmonary disease,Rate,1994,1504.531328,TWN
...,...,...,...,...,...,...,...
41611,Incidence,Niue,Tuberculosis,Rate,2019,39.523161,NIU
41612,Incidence,Niue,Tuberculosis,Rate,2020,39.049782,NIU
41613,Incidence,Niue,Tuberculosis,Rate,2021,38.504013,NIU
41614,Incidence,Niue,Tuberculosis,Rate,2022,37.588972,NIU


In [6]:
countries_to_fix = df_diseases[df_diseases["Country Code"].isna()]["Country Name"].unique()

EXCEPTIONS = {
    # key = name as it appears in dataframe
    # value = correct ISO‑3 alpha‑3 code
    "Democratic People's Republic of Korea": "PRK",
    "Viet Nam":                               "VNM",
    "Micronesia (Federated States of)":       "FSM",
    "Czechia":                                "CZE",
    "North Macedonia":                        "MKD",
    "United States of America":               "USA",
    "Republic of Korea":                      "KOR",
    "Russian Federation":                     "RUS",
    "Republic of Moldova":                    "MDA",
    "Bolivia (Plurinational State of)":       "BOL",
    "Venezuela (Bolivarian Republic of)":     "VEN",
    "Palestine":                              "PSE",  
    "Iran (Islamic Republic of)":             "IRN",
    "Libya":                                  "LBY",
    "Türkiye":                                "TUR",
    "Democratic Republic of the Congo":       "COD",
    "United Republic of Tanzania":            "TZA",
    "Eswatini":                               "SWZ",
    "Cabo Verde":                             "CPV",
    "United States Virgin Islands":           "VIR",
    "South Sudan":                            "SSD"
}

def name_to_iso3(name: str) -> str | None:
    """
    Return the ISO‑3 alpha‑3 code for *name*.
    First tries the EXCEPTIONS dict, then falls back to pycountry.
    Returns None if no match is found.
    """
    # Exception list (covers the non‑standard spellings you gave)
    if name in EXCEPTIONS:
        return EXCEPTIONS[name]

    # Try a direct lookup via pycountry (matches the official ISO name)
    try:
        country = pycountry.countries.lookup(name)
        return country.alpha_3
    except LookupError:
        # No exact match
        return None

# Apply the mapping to the dataframe
for cn in countries_to_fix:
    iso3 = name_to_iso3(cn)

    if iso3 is None:
        print(f"⚠️  No ISO‑3 code found for '{cn}'. Skipping.")
        continue

    df_diseases.loc[
        df_diseases["Country Name"] == cn,
        "Country Code"
    ] = iso3

In [7]:
# different diseases
print(df_diseases["Disease"].unique())

df_COPD = df_diseases[df_diseases["Disease"] == "Chronic obstructive pulmonary disease"]
df_asthma = df_diseases[df_diseases["Disease"] == "Asthma"]
df_tuberculosis = df_diseases[df_diseases["Disease"] == "Tuberculosis"]

['Chronic obstructive pulmonary disease' 'Tuberculosis' 'Asthma']


In [8]:
# combining with different measures
print(df_diseases["Measure"].unique())

df_COPD_prevalence = df_COPD[df_COPD["Measure"] == "Prevalence"]
df_COPD_incidence = df_COPD[df_COPD["Measure"] == "Incidence"]

df_asthma_prevalence = df_asthma[df_asthma["Measure"] == "Prevalence"]
df_asthma_incidence = df_asthma[df_asthma["Measure"] == "Incidence"]

df_tuberculosis_prevalence = df_tuberculosis[df_tuberculosis["Measure"] == "Prevalence"]
df_tuberculosis_incidence = df_tuberculosis[df_tuberculosis["Measure"] == "Incidence"]

['Prevalence' 'Incidence']


In [9]:
# combining with different metrics
print(df_diseases["Metric"].unique())

df_COPD_prevalence_rate = df_COPD_prevalence[df_COPD_prevalence["Metric"] == "Rate"]
df_COPD_incidence_rate = df_COPD_incidence[df_COPD_incidence["Metric"] == "Rate"]

df_asthma_prevalence_rate = df_asthma_prevalence[df_asthma_prevalence["Metric"] == "Rate"]
df_asthma_incidence_rate = df_asthma_incidence[df_asthma_incidence["Metric"] == "Rate"]

df_tuberculosis_prevalence_rate = df_tuberculosis_prevalence[df_tuberculosis_prevalence["Metric"] == "Rate"]
df_tuberculosis_incidence_rate = df_tuberculosis_incidence[df_tuberculosis_incidence["Metric"] == "Rate"]

['Rate']


In [10]:
df_asthma_prevalence_rate[df_asthma_prevalence_rate["Country Name"] == "Greece"]

,Measure,Country Name,Disease,Metric,Year,Value,Country Code
17460,Prevalence,Greece,Asthma,Rate,1990,7133.134059,GRC
17461,Prevalence,Greece,Asthma,Rate,1991,7108.532898,GRC
17462,Prevalence,Greece,Asthma,Rate,1992,7086.240047,GRC
17463,Prevalence,Greece,Asthma,Rate,1993,7072.018062,GRC
17464,Prevalence,Greece,Asthma,Rate,1994,7066.294313,GRC
17465,Prevalence,Greece,Asthma,Rate,1995,7070.735655,GRC
17466,Prevalence,Greece,Asthma,Rate,1996,7087.148514,GRC
17467,Prevalence,Greece,Asthma,Rate,1997,7112.787494,GRC
17468,Prevalence,Greece,Asthma,Rate,1998,7140.928656,GRC
17469,Prevalence,Greece,Asthma,Rate,1999,7169.440991,GRC


## World Developement Indicators Data - 1974 to 2021

In [11]:
df_wdi_1 = pd.read_csv(f"{BASE_PATH}economic/WorldBankGroup/World_Development_Indicators/wdi_1.csv")
df_wdi_2 = pd.read_csv(f"{BASE_PATH}economic/WorldBankGroup/World_Development_Indicators/wdi_2.csv", encoding="cp1252", engine="python")
df_wdi = pd.concat([df_wdi_1, df_wdi_2])
df_wdi.iloc[:-3, :]


,Country Name,Country Code,Series Name,Series Code,1974 [YR1974],1975 [YR1975],1976 [YR1976],1977 [YR1977],1978 [YR1978],1979 [YR1979],...,2012 [YR2012],2013 [YR2013],2014 [YR2014],2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019],2020 [YR2020],2021 [YR2021]
0,Afghanistan,AFG,GDP (current US$),NY.GDP.MKTP.CD,..,..,..,..,..,..,...,19907329777.5872,20146416757.5987,20497128555.6972,19134221644.7325,18116572395.0772,18753456497.8159,18053222687.4126,18799444490.1128,19955929052.1496,14259995441.0759
1,Afghanistan,AFG,GDP per capita (constant 2015 US$),NY.GDP.PCAP.KD,..,..,..,..,..,..,...,568.929021458341,580.603833333096,575.146245808546,565.569730408751,563.872336723147,562.769574140988,553.125151688293,557.861533207459,527.834554499306,408.625855217403
2,Afghanistan,AFG,"Population, total",SP.POP.TOTL,12469127,12773954,13059851,13340756,13611441,13655567,...,30560034,31622704,32792523,33831764,34700612,35688935,36743039,37856121,39068979,40000412
3,Afghanistan,AFG,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,39.469,39.994,40.518,41.082,40.086,38.844,...,61.735,62.188,62.26,62.27,62.646,62.406,62.443,62.941,61.454,60.417
4,Afghanistan,AFG,"Mortality rate, infant (per 1,000 live births)",SP.DYN.IMRT.IN,210.7,207.5,204.1,200.4,196.6,192.9,...,71.3,68.7,66.4,64.2,62.3,60.4,58.6,56.9,55.3,53.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3985,World,WLD,Renewable electricity output (% of total elect...,EG.ELC.RNEW.ZS,24.3995400383505,24.1421558217958,22.2708064372341,21.8056902760591,22.183018520084,22.4624702326194,...,21.2031409070651,21.7907962916827,22.4313850607087,22.9852849250343,23.8879015015538,24.5284507220852,25.1733904780905,26.1929651576066,28.099222284205,27.8784853731492
3986,World,WLD,Access to clean fuels and technologies for coo...,EG.CFT.ACCS.ZS,..,..,..,..,..,..,...,59.5495688485653,60.9543345446539,62.2904088976113,63.6969537121187,65.2272562828093,66.6831170081874,68.2440148869911,69.685504021452,71.1422843390958,72.4370389532721
3987,World,WLD,Access to electricity (% of population),EG.ELC.ACCS.ZS,..,..,..,..,..,..,...,84.9363734839566,85.7108410210463,86.1958012620982,86.9242629695724,88.1047725764967,88.9320882686089,89.797731809362,90.108768771248,90.3960882790908,91.3346473455962
3988,World,WLD,People using at least basic sanitation service...,SH.STA.BASS.ZS,..,..,..,..,..,..,...,..,..,..,..,..,..,..,..,..,..


In [12]:
# Optional: rename the year columns to just the year number
year_cols = [c for c in df_wdi.columns if "[" in c]   # picks the YR… columns
rename_map = {c: c.split("[")[0].strip() for c in year_cols}
df_wdi = df_wdi.rename(columns=rename_map)

print("\nAfter renaming:", df_wdi.columns.tolist(), "\n")

# Melt - Collapse all year columns into one
# Identify the columns that hold the yearly values
year_columns = [c for c in df_wdi.columns if c.isdigit()]   # e.g. ['1974','1975',...]

# Melt (wide → long)
df_long = df_wdi.melt(
    id_vars=['Country Code', 'Series Name', 'Series Code'],
    value_vars=year_columns,
    var_name='Year',          # name of the new column that will hold the year
    value_name='Value'        # name of the column that will hold the measurement
)

print("\nShape after melt:", df_long.shape)

# Pivot - Spread the different series into separate columns
df_tidy = df_long.pivot_table(
    index=['Country Code', 'Year'],   # what defines a unique row
    columns='Series Name',                            # each distinct series becomes a column
    values='Value',                                   # fill cells with the measurement
    aggfunc='first'                                   # there should be only one value per cell
).reset_index()

# After pivot, the column hierarchy is a MultiIndex (Series Names are under the level "Series Name").
# Flatten it for easier use:
df_tidy.columns.name = None          # drop the name of the columns axis
df_tidy = df_tidy.rename_axis(None, axis=1)   # also removes the axis name

print("\nFinal shape:", df_tidy.shape)
df_wdi = df_tidy
df_wdi


After renaming: ['Country Name', 'Country Code', 'Series Name', 'Series Code', '1974', '1975', '1976', '1977', '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986', '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021'] 


Shape after melt: (383424, 5)

Final shape: (12768, 17)


,Country Code,Year,Access to clean fuels and technologies for cooking (% of population),Access to electricity (% of population),Carbon dioxide (CO2) emissions excluding LULUCF per capita (t CO2e/capita),"Compulsory education, duration (years)",GDP (current US$),GDP per capita (constant 2015 US$),Gini index,"Life expectancy at birth, total (years)","Mortality rate, infant (per 1,000 live births)",People using at least basic sanitation services (% of population),"Population, total",Poverty headcount ratio at national poverty lines (% of population),Renewable electricity output (% of total electricity output),Surface area (sq. km),"Unemployment, total (% of total labor force) (national estimate)"
0,ABW,1974,..,..,0.745514061937651,..,..,..,..,69.278,..,..,58349,..,..,180,..
1,ABW,1975,..,..,0.984647053778197,..,..,..,..,69.564,..,..,58295,..,..,180,..
2,ABW,1976,..,..,0.966282894736842,..,..,..,..,69.808,..,..,58368,..,..,180,..
3,ABW,1977,..,..,1.14544213041994,..,..,..,..,70.054,..,..,58580,..,..,180,..
4,ABW,1978,..,..,1.22328841704097,..,..,..,..,70.271,..,..,58776,..,..,180,..
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12763,ZWE,2017,30.1,44,0.714627028745081,7,51074726484.0037,1422.1934603003,44.3,60.263,49.2,36.9416742711943,14812482,30.4,55.1814329227917,390760,..
12764,ZWE,2018,30.3,45.4,0.816125522899006,7,34156057417.3285,1471.39488971183,..,60.906,47.4,36.3571601293685,15034452,..,63.0333013128402,390760,..
12765,ZWE,2019,30.3,46.7,0.731381759643275,7,25715657177.4682,1356.83821089692,50.3,61.06,46,35.7743358079873,15271368,38.3,68.8452182208443,390760,7.373
12766,ZWE,2020,30.5,52.7,0.584283212450557,7,26868564055.12,1230.19155671068,..,61.53,44.9,35.1923618234591,15526888,..,60.7855537239622,390760,..


In [13]:
df_wdi["Country Code"].unique()

array(['ABW', 'AFE', 'AFG', 'AFW', 'AGO', 'ALB', 'AND', 'ARB', 'ARE',
       'ARG', 'ARM', 'ASM', 'ATG', 'AUS', 'AUT', 'AZE', 'BDI', 'BEL',
       'BEN', 'BFA', 'BGD', 'BGR', 'BHR', 'BHS', 'BIH', 'BLR', 'BLZ',
       'BMU', 'BOL', 'BRA', 'BRB', 'BRN', 'BTN', 'BWA', 'CAF', 'CAN',
       'CEB', 'CHE', 'CHI', 'CHL', 'CHN', 'CIV', 'CMR', 'COD', 'COG',
       'COL', 'COM', 'CPV', 'CRI', 'CSS', 'CUB', 'CUW', 'CYM', 'CYP',
       'CZE', 'DEU', 'DJI', 'DMA', 'DNK', 'DOM', 'DZA', 'EAP', 'EAR',
       'EAS', 'ECA', 'ECS', 'ECU', 'EGY', 'EMU', 'ERI', 'ESP', 'EST',
       'ETH', 'EUU', 'FCS', 'FIN', 'FJI', 'FRA', 'FRO', 'FSM', 'GAB',
       'GBR', 'GEO', 'GHA', 'GIB', 'GIN', 'GMB', 'GNB', 'GNQ', 'GRC',
       'GRD', 'GRL', 'GTM', 'GUM', 'GUY', 'HIC', 'HKG', 'HND', 'HPC',
       'HRV', 'HTI', 'HUN', 'IBD', 'IBT', 'IDA', 'IDB', 'IDN', 'IDX',
       'IMN', 'IND', 'INX', 'IRL', 'IRN', 'IRQ', 'ISL', 'ISR', 'ITA',
       'JAM', 'JOR', 'JPN', 'KAZ', 'KEN', 'KGZ', 'KHM', 'KIR', 'KNA',
       'KOR', 'KWT',

## CO2 emissions per capita

In [14]:
df_co2_emissions = pd.read_csv(f"{BASE_PATH}gapminder/c_emission_cap.csv")

# Melt - Collapse all year columns into one
# Identify the columns that hold the yearly values
year_columns = [c for c in df_co2_emissions.columns if c.isdigit()] 

# Melt (wide → long)
df_co2_emissions = df_co2_emissions.melt(
    id_vars=['geo'],
    value_vars=year_columns,
    var_name='Year',          # name of the new column that will hold the year
    value_name='Value'        # name of the column that will hold the measurement
)

df_co2_emissions.columns = ['Country Code', 'Year', 'CO2 emissions']
df_co2_emissions

,Country Code,Year,CO2 emissions
0,alb,1990,1.67653
1,are,1990,29.29966
2,arg,1990,3.41425
3,arm,1990,1.67386
4,aus,1990,14.18586
...,...,...,...
3653,ven,2020,0.63081
3654,vnm,2020,2.09643
3655,zaf,2020,5.11405
3656,zmb,2020,0.38845


## Coal consumptions per capita

In [15]:
df_coal_consumption = pd.read_csv(f"{BASE_PATH}gapminder/coal_consumption_per_cap.csv")

# Melt - Collapse all year columns into one
# Identify the columns that hold the yearly values
year_columns = [c for c in df_coal_consumption.columns if c.isdigit()] 

# Melt (wide → long)
df_coal_consumption = df_coal_consumption.melt(
    id_vars=['geo'],
    value_vars=year_columns,
    var_name='Year',          # name of the new column that will hold the year
    value_name='Value'        # name of the column that will hold the measurement
)

df_coal_consumption.columns = ['Country Code','Year', 'Coal consumption']
df_coal_consumption

,Country Code,Year,Coal consumption
0,are,1965,NaN
1,arg,1965,0.03281
2,aus,1965,1.53549
3,aut,1965,0.69581
4,aze,1965,NaN
...,...,...,...
4230,usa,2019,0.82314
4231,uzb,2019,0.05264
4232,ven,2019,0.00265
4233,vnm,2019,0.51254


## Energy use per person

In [16]:
df_energy_use = pd.read_csv(f"{BASE_PATH}gapminder/energy_use_per_person.csv")

# Threshold: keep columns with at least 60% non-NaN values
threshold = 0.6 * len(df_energy_use)

# Drop columns with less than threshold non-NaN values
df_energy_use = df_energy_use.dropna(axis=1, thresh=threshold)

# Melt - Collapse all year columns into one
# Identify the columns that hold the yearly values
year_columns = [c for c in df_energy_use.columns if c.isdigit()] 

# Melt (wide → long)
df_energy_use = df_energy_use.melt(
    id_vars=['geo'],
    value_vars=year_columns,
    var_name='Year',          # name of the new column that will hold the year
    value_name='Value'        # name of the column that will hold the measurement
)

df_energy_use.columns = ['Country Code', 'Year', 'Energy use']
df_energy_use

,Country Code,Year,Energy use
0,ago,1971,642.74452
1,alb,1971,785.16153
2,are,1971,3020.53267
3,arg,1971,1387.13973
4,arm,1971,NaN
...,...,...,...
9095,wsm,2022,NaN
9096,yem,2022,68.94953
9097,zaf,2022,1976.60892
9098,zmb,2022,800.67487


## Electricity use per person

In [17]:
df_electricity = pd.read_csv(f"{BASE_PATH}gapminder/electricity_use_per_person.csv")

# Threshold: keep columns with at least 60% non-NaN values
threshold = 0.6 * len(df_electricity)

# Drop columns with less than threshold non-NaN values
df_electricity = df_electricity.dropna(axis=1, thresh=threshold)

# Melt - Collapse all year columns into one
# Identify the columns that hold the yearly values
year_columns = [c for c in df_electricity.columns if c.isdigit()] 

# Melt (wide → long)
df_electricity = df_electricity.melt(
    id_vars=['geo'],
    value_vars=year_columns,
    var_name='Year',          # name of the new column that will hold the year
    value_name='Value'        # name of the column that will hold the measurement
)

df_electricity.columns = ['Country Code', 'Year', 'Electricity use']
df_electricity

,Country Code,Year,Electricity use
0,ago,1971,92.80430
1,alb,1971,532.02843
2,are,1971,564.13364
3,arg,1971,874.54418
4,arm,1971,NaN
...,...,...,...
7587,vnm,2022,2624.40089
7588,yem,2022,53.97291
7589,zaf,2022,3357.92464
7590,zmb,2022,703.96684


## Material footprint per capita

In [18]:
df_mat_foot_pcap = pd.read_csv(f"{BASE_PATH}gapminder/matfootp_cap.csv")

# Threshold: keep columns with at least 60% non-NaN values
threshold = 0.6 * len(df_mat_foot_pcap)

# Drop columns with less than threshold non-NaN values
df_mat_foot_pcap = df_mat_foot_pcap.dropna(axis=1, thresh=threshold)

# Melt - Collapse all year columns into one
# Identify the columns that hold the yearly values
year_columns = [c for c in df_mat_foot_pcap.columns if c.isdigit()] 

# Melt (wide → long)
df_mat_foot_pcap = df_mat_foot_pcap.melt(
    id_vars=['geo'],
    value_vars=year_columns,
    var_name='Year',          # name of the new column that will hold the year
    value_name='Value'        # name of the column that will hold the measurement
)

df_mat_foot_pcap.columns = ['Country Code', 'Year', 'Raw materials extracted']
df_mat_foot_pcap

,Country Code,Year,Raw materials extracted
0,afg,1970,2.34
1,ago,1970,3.82
2,alb,1970,5.61
3,are,1970,224.22
4,arg,1970,14.11
...,...,...,...
7895,vnm,2019,8.52
7896,yem,2019,2.89
7897,zaf,2019,7.55
7898,zmb,2019,2.38


## Natural gas production per person

In [19]:
df_natural_gas = pd.read_csv(f"{BASE_PATH}gapminder/natural_gas_production_per_person.csv")

# Threshold: keep columns with at least 60% non-NaN values
threshold = 0.6 * len(df_natural_gas)

# Drop columns with less than threshold non-NaN values
df_natural_gas = df_natural_gas.dropna(axis=1, thresh=threshold)

# Melt - Collapse all year columns into one
# Identify the columns that hold the yearly values
year_columns = [c for c in df_natural_gas.columns if c.isdigit()] 

# Melt (wide → long)
df_natural_gas = df_natural_gas.melt(
    id_vars=['geo'],
    value_vars=year_columns,
    var_name='Year',          # name of the new column that will hold the year
    value_name='Value'        # name of the column that will hold the measurement
)

df_natural_gas.columns = ['Country Code', 'Year', 'Natural gas production']
df_natural_gas

,Country Code,Year,Natural gas production
0,are,1970,2.97784
1,arg,1970,0.21080
2,aus,1970,0.11630
3,aze,1970,NaN
4,bgd,1970,NaN
...,...,...,...
2445,usa,2019,2.40626
2446,uzb,2019,1.46707
2447,ven,2019,0.79773
2448,vnm,2019,0.08783


In [20]:
duplicates = df_natural_gas.groupby(['Country Code', 'Year'])['Natural gas production'].nunique()
duplicates = duplicates[duplicates > 1]
duplicates

Series([], Name: Natural gas production, dtype: int64)

## Oil consumption per capita

In [21]:
df_oil_consumption = pd.read_csv(f"{BASE_PATH}gapminder/oil_consumption_per_cap.csv")

# Threshold: keep columns with at least 60% non-NaN values
threshold = 0.6 * len(df_oil_consumption)

# Drop columns with less than threshold non-NaN values
df_oil_consumption = df_oil_consumption.dropna(axis=1, thresh=threshold)

# Melt - Collapse all year columns into one
# Identify the columns that hold the yearly values
year_columns = [c for c in df_oil_consumption.columns if c.isdigit()] 

# Melt (wide → long)
df_oil_consumption = df_oil_consumption.melt(
    id_vars=['geo'],
    value_vars=year_columns,
    var_name='Year',          # name of the new column that will hold the year
    value_name='Value'        # name of the column that will hold the measurement
)

df_oil_consumption.columns = ['Country Code','Year', 'Oil consumption']
df_oil_consumption

,Country Code,Year,Oil consumption
0,are,1965,0.46424
1,arg,1965,0.99504
2,aus,1965,1.33667
3,aut,1965,0.76124
4,aze,1965,NaN
...,...,...,...
4230,usa,2019,2.55812
4231,uzb,2019,0.06328
4232,ven,2019,0.56686
4233,vnm,2019,0.25480


## Sulfur emission

In [22]:
df_sulfur_emissions = pd.read_csv(f"{BASE_PATH}gapminder/sulfur_emissions_per_person_kg.csv")

# Threshold: keep columns with at least 60% non-NaN values
threshold = 0.6 * len(df_sulfur_emissions)

# Drop columns with less than threshold non-NaN values
df_sulfur_emissions = df_sulfur_emissions.dropna(axis=1, thresh=threshold)

# Melt - Collapse all year columns into one
# Identify the columns that hold the yearly values
year_columns = [c for c in df_sulfur_emissions.columns if c.isdigit()] 

# Melt (wide → long)
df_sulfur_emissions = df_sulfur_emissions.melt(
    id_vars=['geo'],
    value_vars=year_columns,
    var_name='Year',          # name of the new column that will hold the year
    value_name='Value'        # name of the column that will hold the measurement
)

df_sulfur_emissions.columns = ['Country Code', 'Year', 'Sulfur emission']
df_sulfur_emissions

,Country Code,Year,Sulfur emission
0,afg,1850,NaN
1,ago,1850,0.00003
2,alb,1850,0.00017
3,are,1850,NaN
4,arg,1850,0.00466
...,...,...,...
24759,vnm,2000,1.12400
24760,yem,2000,1.91900
24761,zaf,2000,35.02000
24762,zmb,2000,42.62000


## Nuclear power generation per person

In [23]:
df_nuclear_power_gen = pd.read_csv(f"{BASE_PATH}gapminder/nuclear_power_generation_per_person.csv")

# Threshold: keep columns with at least 60% non-NaN values
threshold = 0.6 * len(df_nuclear_power_gen)

# Drop columns with less than threshold non-NaN values
df_nuclear_power_gen = df_nuclear_power_gen.dropna(axis=1, thresh=threshold)

# Melt - Collapse all year columns into one
# Identify the columns that hold the yearly values
year_columns = [c for c in df_nuclear_power_gen.columns if c.isdigit()] 

# Melt (wide → long)
df_nuclear_power_gen = df_nuclear_power_gen.melt(
    id_vars=['geo'],
    value_vars=year_columns,
    var_name='Year',          # name of the new column that will hold the year
    value_name='Value'        # name of the column that will hold the measurement
)

df_nuclear_power_gen.columns = ['Country Code', 'Year', 'Nuclear power generation']
df_nuclear_power_gen

,Country Code,Year,Nuclear power generation
0,arg,1977,0.00523
1,arm,1977,NaN
2,bel,1977,0.10450
3,bgr,1977,0.05760
4,bra,1977,NaN
...,...,...,...
1018,svn,2010,0.23970
1019,swe,2010,0.53010
1020,ukr,2010,0.16870
1021,usa,2010,0.23240


## Prevalence of current tobacco use

In [24]:
df_tobacco_use = pd.read_csv(f"{BASE_PATH}gapminder/sh_prv_smok.csv")

# Threshold: keep columns with at least 60% non-NaN values
threshold = 0.6 * len(df_tobacco_use)

# Drop columns with less than threshold non-NaN values
#df_tobacco_use = df_tobacco_use.dropna(axis=1, thresh=threshold)

# Melt - Collapse all year columns into one
# Identify the columns that hold the yearly values
year_columns = [c for c in df_tobacco_use.columns if c.isdigit()] 

# Melt (wide → long)
df_tobacco_use = df_tobacco_use.melt(
    id_vars=['geo'],
    value_vars=year_columns,
    var_name='Year',          # name of the new column that will hold the year
    value_name='Value'        # name of the column that will hold the measurement
)

df_tobacco_use.columns = ['Country Code', 'Year', 'Prevalence tobacco use']
df_tobacco_use

,Country Code,Year,Prevalence tobacco use
0,afg,2000,39.2
1,alb,2000,35.4
2,and,2000,36.0
3,are,2000,19.1
4,arg,2000,34.3
...,...,...,...
3767,wsm,2022,22.5
3768,yem,2022,21.4
3769,zaf,2022,20.7
3770,zmb,2022,14.6


## Air pollution PM 2.5

In [25]:
df_air_pollution = pd.read_csv(f"{BASE_PATH}gapminder/air_pollution_pm25.csv")

# Threshold: keep columns with at least 60% non-NaN values
threshold = 0.6 * len(df_air_pollution)

# Drop columns with less than threshold non-NaN values
df_air_pollution = df_air_pollution.dropna(axis=1, thresh=threshold)

# Melt - Collapse all year columns into one
# Identify the columns that hold the yearly values
year_columns = [c for c in df_air_pollution.columns if c.isdigit()] 

# Melt (wide → long)
df_air_pollution = df_air_pollution.melt(
    id_vars=['geo'],
    value_vars=year_columns,
    var_name='Year',          # name of the new column that will hold the year
    value_name='Value'        # name of the column that will hold the measurement
)

df_air_pollution.columns = ['Country Code', 'Year', 'PM2.5 pollution']
df_air_pollution

,Country Code,Year,PM2.5 pollution
0,afg,1990,64.17410
1,ago,1990,23.83177
2,alb,1990,22.96158
3,and,1990,16.82719
4,are,1990,44.29661
...,...,...,...
5947,wsm,2020,12.55190
5948,yem,2020,34.83236
5949,zaf,2020,23.75307
5950,zmb,2020,24.30859


## Physicians per 1000 people

In [26]:
df_physicians = pd.read_csv(f"{BASE_PATH}gapminder/sh_med_phys_zs.csv")

# Threshold: keep columns with at least 60% non-NaN values
threshold = 0.6 * len(df_physicians)

# Drop columns with less than threshold non-NaN values
#df_physicians = df_physicians.dropna(axis=1, thresh=threshold)

# Melt - Collapse all year columns into one
# Identify the columns that hold the yearly values
year_columns = [c for c in df_physicians.columns if c.isdigit()] 

# Melt (wide → long)
df_physicians = df_physicians.melt(
    id_vars=['geo'],
    value_vars=year_columns,
    var_name='Year',          # name of the new column that will hold the year
    value_name='Value'        # name of the column that will hold the measurement
)

df_physicians.columns = ['Country Code', 'Year', 'Physicians per 1000']
df_physicians

,Country Code,Year,Physicians per 1000
0,afg,1960,0.035
1,ago,1960,0.067
2,alb,1960,0.276
3,and,1960,NaN
4,are,1960,NaN
...,...,...,...
12091,wsm,2022,NaN
12092,yem,2022,NaN
12093,zaf,2022,NaN
12094,zmb,2022,0.262


## Population aged 60+ (%)

In [27]:
df_pop_60plus = pd.read_csv(f"{BASE_PATH}gapminder/population_aged_60plus_years_both_sexes_percent.csv")

# Threshold: keep columns with at least 60% non-NaN values
threshold = 0.6 * len(df_pop_60plus)

# Drop columns with less than threshold non-NaN values
#df_pop_60plus = df_pop_60plus.dropna(axis=1, thresh=threshold)

# Melt - Collapse all year columns into one
# Identify the columns that hold the yearly values
year_columns = [c for c in df_pop_60plus.columns if c.isdigit()] 

# Melt (wide → long)
df_pop_60plus = df_pop_60plus.melt(
    id_vars=['geo'],
    value_vars=year_columns,
    var_name='Year',          # name of the new column that will hold the year
    value_name='Value'        # name of the column that will hold the measurement
)

df_pop_60plus.columns = ['Country Code', 'Year', 'Population 60+ (%)']
df_pop_60plus

,Country Code,Year,Population 60+ (%)
0,afg,1950,5.00529
1,ago,1950,4.75912
2,alb,1950,8.89032
3,and,1950,14.55971
4,are,1950,5.51455
...,...,...,...
29440,wsm,2100,21.98891
29441,yem,2100,21.85242
29442,zaf,2100,24.13136
29443,zmb,2100,19.40224


## Population aged 40-59 (%)

In [28]:
df_pop_40_59 = pd.read_csv(f"{BASE_PATH}gapminder/population_aged_40_59_years_both_sexes_percent.csv")

# Threshold: keep columns with at least 60% non-NaN values
threshold = 0.6 * len(df_pop_40_59)

# Drop columns with less than threshold non-NaN values
#df_pop_40_59 = df_pop_40_59.dropna(axis=1, thresh=threshold)

# Melt - Collapse all year columns into one
# Identify the columns that hold the yearly values
year_columns = [c for c in df_pop_40_59.columns if c.isdigit()] 

# Melt (wide → long)
df_pop_40_59 = df_pop_40_59.melt(
    id_vars=['geo'],
    value_vars=year_columns,
    var_name='Year',          # name of the new column that will hold the year
    value_name='Value'        # name of the column that will hold the measurement
)

df_pop_40_59.columns = ['Country Code', 'Year', 'Population 40-59 (%)']
df_pop_40_59

,Country Code,Year,Population 40-59 (%)
0,afg,1950,15.08720
1,ago,1950,14.77321
2,alb,1950,15.30447
3,and,1950,25.18346
4,are,1950,14.32118
...,...,...,...
29440,wsm,2100,24.31998
29441,yem,2100,25.67912
29442,zaf,2100,24.88155
29443,zmb,2100,24.36125


## Industry workers (% of employment)

In [29]:
df_industry_workers = pd.read_csv(f"{BASE_PATH}gapminder/industry_workers_percent_of_employment.csv")

# Threshold: keep columns with at least 60% non-NaN values
threshold = 0.6 * len(df_industry_workers)

# Drop columns with less than threshold non-NaN values
#df_industry_workers = df_industry_workers.dropna(axis=1, thresh=threshold)

# Melt - Collapse all year columns into one
# Identify the columns that hold the yearly values
year_columns = [c for c in df_industry_workers.columns if c.isdigit()] 

# Melt (wide → long)
df_industry_workers = df_industry_workers.melt(
    id_vars=['geo'],
    value_vars=year_columns,
    var_name='Year',          # name of the new column that will hold the year
    value_name='Value'        # name of the column that will hold the measurement
)

df_industry_workers.columns = ['Country Code', 'Year', 'Industry workers (%)']
df_industry_workers

,Country Code,Year,Industry workers (%)
0,afg,1991,8.98493
1,ago,1991,8.31515
2,alb,1991,10.85359
3,are,1991,29.31161
4,arg,1991,27.84588
...,...,...,...
5902,wsm,2023,13.78016
5903,yem,2023,11.22562
5904,zaf,2023,17.69290
5905,zmb,2023,9.86643


## Agriculture workers (% of employment)

In [30]:
df_agriculture_workers = pd.read_csv(f"{BASE_PATH}gapminder/agriculture_workers_percent_of_employment.csv")

# Threshold: keep columns with at least 60% non-NaN values
threshold = 0.6 * len(df_agriculture_workers)

# Drop columns with less than threshold non-NaN values
#df_agriculture_workers = df_agriculture_workers.dropna(axis=1, thresh=threshold)

# Melt - Collapse all year columns into one
# Identify the columns that hold the yearly values
year_columns = [c for c in df_agriculture_workers.columns if c.isdigit()] 

# Melt (wide → long)
df_agriculture_workers = df_agriculture_workers.melt(
    id_vars=['geo'],
    value_vars=year_columns,
    var_name='Year',          # name of the new column that will hold the year
    value_name='Value'        # name of the column that will hold the measurement
)

df_agriculture_workers.columns = ['Country Code', 'Year', 'Agriculture workers (%)']
df_agriculture_workers

,Country Code,Year,Agriculture workers (%)
0,afg,1991,68.04182
1,ago,1991,39.82865
2,alb,1991,53.72382
3,are,1991,9.31670
4,arg,1991,15.66901
...,...,...,...
5902,wsm,2023,30.21310
5903,yem,2023,28.55788
5904,zaf,2023,18.79308
5905,zmb,2023,55.44526


## Land Use - OECD

In [31]:
df_land_use = pd.read_csv(f"{BASE_PATH}enivronment/OECD/land_use.csv")
measure_col = df_land_use.pivot(columns='Measure', values='OBS_VALUE')
df_land_use = pd.concat([df_land_use, measure_col], axis = 1)
df_land_use = df_land_use[["REF_AREA", "TIME_PERIOD", "Total area"]] #unit measure is Square Km for all
df_land_use.columns = ["Country Code", "Year", "Land use total area (Square Km)"]
df_land_use

,Country Code,Year,Land use total area (Square Km)
0,SHN,2010,390.0
1,SHN,2011,390.0
2,SHN,2012,390.0
3,SHN,2013,390.0
4,SHN,2014,390.0
...,...,...,...
3211,ZWE,2019,390760.0
3212,ZWE,2020,390760.0
3213,ZWE,2021,390760.0
3214,ZWE,2022,390760.0


In [32]:
exclude_esnemble_countries = ['OECD', 'OECDE', 'OECDA', 'OECDSO']
df_land_use = df_land_use[~df_land_use['Country Code'].isin(exclude_esnemble_countries)]
df_land_use["Country Code"].unique()

array(['SHN', 'MNP', 'PLW', 'SYC', 'GIB', 'TKL', 'COK', 'SPM', 'DMA',
       'TON', 'NIU', 'KNA', 'VCT', 'NRU', 'ABW', 'MHL', 'VIR', 'AUS',
       'AUT', 'BEL', 'CAN', 'CHL', 'COL', 'CRI', 'CZE', 'DNK', 'EST',
       'FIN', 'FRA', 'DEU', 'GRC', 'HUN', 'ISL', 'IRL', 'ISR', 'ITA',
       'JPN', 'KOR', 'LVA', 'LTU', 'LUX', 'MEX', 'NLD', 'NZL', 'NOR',
       'POL', 'PRT', 'SVK', 'SVN', 'ESP', 'SWE', 'CHE', 'TUR', 'GBR',
       'USA', 'AFG', 'ALB', 'DZA', 'ASM', 'AGO', 'AND', 'AIA', 'ATG',
       'ARG', 'ARM', 'AZE', 'BHS', 'BHR', 'BGD', 'BRB', 'BLR', 'BLZ',
       'BEN', 'BMU', 'BTN', 'BOL', 'BIH', 'BWA', 'BRA', 'VGB', 'BRN',
       'BGR', 'BFA', 'BDI', 'CPV', 'KHM', 'CMR', 'CYM', 'CAF', 'TCD',
       'CHN', 'COM', 'COG', 'CIV', 'HRV', 'CUB', 'CUW', 'CYP', 'PRK',
       'COD', 'DJI', 'DOM', 'ECU', 'EGY', 'SLV', 'GNQ', 'ERI', 'SWZ',
       'ETH', 'FLK', 'FRO', 'FJI', 'GUF', 'PYF', 'GAB', 'GMB', 'GEO',
       'GHA', 'GRL', 'GRD', 'GLP', 'GUM', 'GTM', 'GIN', 'GNB', 'GUY',
       'HTI', 'VAT',

## Meteo - era5

In [ ]:
df_meteo_data = pd.read_csv(f"{BASE_PATH}enivronment/era5/era5_climate_country.csv")
df_meteo_data

,Year,Country Code,u10,v10,d2m,t2m,sst,sp,skt,blh
0,1980,IDN,-0.731723,0.000275,295.007338,298.317703,34.691068,98360.698946,298.493671,426.010201
1,1980,MYS,0.317648,0.064888,294.381366,297.039004,71.816384,96500.498069,297.242948,342.285277
2,1980,CYP,0.641971,0.409293,285.697660,289.763028,290.739371,102027.155390,290.599272,867.391694
3,1980,IND,-0.736435,0.067963,288.873006,295.436724,149.975586,94774.793878,296.593739,629.126892
4,1980,CHN,0.685689,0.316234,283.616753,288.801672,166.547391,99887.850986,289.777521,703.330091
5,1980,ISR,-1.263409,-0.135346,287.224559,291.596025,292.759796,102050.323758,292.589532,876.032102
6,1980,PSE,-1.046189,-0.083186,287.067098,291.401244,292.530232,102058.536818,292.364343,872.993833
7,1980,LBN,0.054841,0.075855,286.157807,290.317094,291.317997,102061.355379,291.172199,870.135161
8,1980,ETH,-5.125968,-0.608600,295.938907,298.947593,300.512386,101070.522154,300.333512,637.957981
9,1980,SSD,-5.213069,-0.429473,296.392848,299.271879,300.985183,100999.498753,300.798523,611.687000


## Pesticides Use - OECD

In [33]:
df_pesticides_use = pd.read_csv(f"{BASE_PATH}enivronment/OECD/pesticides_use.csv")
measure_col = df_pesticides_use.pivot(columns='Measure', values='OBS_VALUE')[["Total molluscicides", "Total sales of agricultural pesticides"]]
df_pesticides_use = pd.concat([df_pesticides_use, measure_col], axis = 1)

df_pesticides_use_total_pesticides = df_pesticides_use[df_pesticides_use["Total sales of agricultural pesticides"].notnull()][["REF_AREA", "TIME_PERIOD", "Total sales of agricultural pesticides"]]
df_pesticides_use_total_molluscicides = df_pesticides_use[df_pesticides_use["Total molluscicides"].notnull()][["REF_AREA", "TIME_PERIOD", "Total molluscicides"]]
df_pesticides_use_total_pesticides.columns = ["Country Code", "Year", "Total sales of agricultural pesticides (tonnes)"]
df_pesticides_use_total_molluscicides.columns = ["Country Code", "Year", "Total molluscicides (tonnes)"]
display(df_pesticides_use_total_pesticides)
display(df_pesticides_use_total_molluscicides)

,Country Code,Year,Total sales of agricultural pesticides (tonnes)
0,AUS,2012,48687.875
1,AUS,2013,45177.187
2,AUS,2014,49857.349
3,AUS,2015,50921.602
4,AUS,2016,63416.482
...,...,...,...
481,VNM,2016,19154.000
482,VNM,2017,19154.000
483,VNM,2018,19154.000
484,VNM,2019,19154.000


,Country Code,Year,Total molluscicides (tonnes)
2000,AUT,2012,23.653
2001,AUT,2013,13.471
2002,AUT,2014,16.180
2003,AUT,2015,21.214
2004,AUT,2016,10.679
...,...,...,...
2240,ROU,2017,4.981
2241,ROU,2018,4.829
2242,ROU,2019,4.263
2243,ROU,2020,9.304


In [34]:
df_pesticides_use_total_pesticides["Country Code"].unique()

array(['AUS', 'AUT', 'BEL', 'CAN', 'CHL', 'COL', 'CRI', 'CZE', 'DNK',
       'EST', 'FIN', 'FRA', 'DEU', 'GRC', 'HUN', 'ISL', 'IRL', 'ISR',
       'ITA', 'JPN', 'KOR', 'LVA', 'LTU', 'LUX', 'MEX', 'NLD', 'NOR',
       'POL', 'PRT', 'SVK', 'SVN', 'ESP', 'SWE', 'CHE', 'TUR', 'GBR',
       'USA', 'ARG', 'BRA', 'BGR', 'CHN', 'HRV', 'CYP', 'IND', 'IDN',
       'KAZ', 'MLT', 'ROU', 'RUS', 'ZAF', 'UKR', 'VNM'], dtype=object)

## Putting data together

In [35]:
def inner_merge(df1, df2):
    merged = pd.merge(
        df1, df2,
        on=["Country Code", "Year"],
        how="outer"
    )
    return merged

In [36]:
df_wdi = df_wdi.astype({'Year': 'string'})
df_co2_emissions = df_co2_emissions.astype({'Year': 'string'})
df_air_pollution = df_air_pollution.astype({'Year': 'string'})
df_coal_consumption = df_coal_consumption.astype({'Year': 'string'})
df_energy_use = df_energy_use.astype({'Year': 'string'})
df_electricity = df_electricity.astype({'Year': 'string'})
df_mat_foot_pcap = df_mat_foot_pcap.astype({'Year': 'string'})
df_natural_gas = df_natural_gas.astype({'Year': 'string'})
df_oil_consumption = df_oil_consumption.astype({'Year': 'string'})
df_sulfur_emissions = df_sulfur_emissions.astype({'Year': 'string'})
df_nuclear_power_gen = df_nuclear_power_gen.astype({'Year': 'string'})
df_tobacco_use = df_tobacco_use.astype({'Year': 'string'})
df_pesticides_use_total_pesticides = df_pesticides_use_total_pesticides.astype({'Year': 'string'})
df_meteo_data = df_meteo_data.astype({'Year': 'string'})
df_physicians = df_physicians.astype({'Year': 'string'})
df_pop_60plus = df_pop_60plus.astype({'Year': 'string'})
df_pop_40_59 = df_pop_40_59.astype({'Year': 'string'})
df_industry_workers = df_industry_workers.astype({'Year': 'string'})
df_agriculture_workers = df_agriculture_workers.astype({'Year': 'string'})

dataframes = [df_wdi, df_co2_emissions, df_coal_consumption, df_energy_use, df_electricity, df_mat_foot_pcap, df_natural_gas,\
              df_oil_consumption, df_sulfur_emissions, df_nuclear_power_gen, df_tobacco_use, df_pesticides_use_total_pesticides,\
              df_meteo_data, df_air_pollution, df_physicians, df_pop_60plus, df_pop_40_59, df_industry_workers, df_agriculture_workers]

for df in dataframes:
    df['Country Code'] = df['Country Code'].str.strip().str.upper()
    df['Year'] = df['Year'].astype(str).str.strip()

# Apply smart_merge cumulatively
df_merged = reduce(inner_merge, dataframes)

# Compute completeness
non_key_cols = [c for c in df_merged.columns if c not in ["Country Code", "Year"]]
df_merged["data_completeness"] = df_merged[non_key_cols].notna().mean(axis=1)

threshold = 0.6
df_filtered = df_merged[df_merged["data_completeness"] >= threshold]

print(f"Keeping {len(df_filtered)} rows with >= {threshold*100:.0f}% completeness")

columns_to_keep = [col for col in df_filtered.columns if col not in ['data_completeness']]
df_filtered = df_filtered[columns_to_keep]

print(df_filtered['Year'].unique())
print(df_filtered['Country Code'].unique())

Keeping 6876 rows with >= 60% completeness
['1980' '1981' '1982' '1983' '1984' '1985' '1986' '1987' '1988' '1989'
 '1990' '1991' '1992' '1993' '1994' '1995' '1996' '1997' '1998' '1999'
 '2000' '2001' '2002' '2003' '2004' '2005' '2006' '2007' '2008' '2009'
 '2010' '2011' '2012' '2013' '2014' '2015' '2016' '2017' '2018' '2019'
 '2020' '2021' '1974' '1975' '1976' '1977' '1978' '1979']
['AFG' 'AGO' 'ALB' 'AND' 'ARE' 'ARG' 'ARM' 'AUS' 'AUT' 'AZE' 'BDI' 'BEL'
 'BEN' 'BFA' 'BGD' 'BGR' 'BHR' 'BIH' 'BLR' 'BOL' 'BRA' 'BRN' 'BTN' 'BWA'
 'CAF' 'CAN' 'CHE' 'CHL' 'CHN' 'CIV' 'CMR' 'COD' 'COG' 'COL' 'COM' 'CRI'
 'CUB' 'CYP' 'CZE' 'DEU' 'DJI' 'DNK' 'DOM' 'DZA' 'ECU' 'EGY' 'ERI' 'ESP'
 'EST' 'ETH' 'FIN' 'FJI' 'FRA' 'GAB' 'GBR' 'GEO' 'GHA' 'GIN' 'GMB' 'GNQ'
 'GRC' 'GTM' 'HND' 'HRV' 'HTI' 'HUN' 'IDN' 'IND' 'IRL' 'IRN' 'IRQ' 'ISL'
 'ISR' 'ITA' 'JAM' 'JOR' 'JPN' 'KAZ' 'KEN' 'KGZ' 'KHM' 'KOR' 'KWT' 'LAO'
 'LBN' 'LBY' 'LKA' 'LSO' 'LTU' 'LUX' 'LVA' 'MAR' 'MDA' 'MDG' 'MEX' 'MKD'
 'MLI' 'MLT' 'MMR' 'MNE' 'MNG' 

## Final datasets creation, initial filtering and saving

In [37]:
def filter_high_nan(df, group_col, threshold=0.6):
    """
    Remove groups in `group_col` where the average NaN ratio across other columns > threshold.
    Returns filtered DataFrame and list of dropped groups.
    """
    other_cols = [col for col in df.columns if col != group_col]
    nan_ratio = (
        df[other_cols]
        .groupby(df[group_col], observed=True)
        .apply(lambda group: group.isna().mean().mean())
    )
    groups_to_drop = nan_ratio[nan_ratio > threshold].index.tolist()
    df_filtered = df[~df[group_col].isin(groups_to_drop)]
    return df_filtered, groups_to_drop

In [ ]:
def save_final_csv(df_left, df_right, df_name):
    df_left['Year'] = df_left['Year'].astype(str)

    # Merge
    df_merge = pd.merge(
        df_left, df_right,
        on=["Country Code", "Year"],
        how="inner",
        suffixes=("_1", "_2")
    )

    # On calcule le ratio de manquants mais on ne supprime PAS aveuglément
    missing_ratio = df_merge.isna().mean()
    
    # On définit une liste de colonnes "Critiques" à garder quoi qu'il arrive
    critical_columns = [
        'Prevalence tobacco use', 
        'Sulfur emission',
        'Agriculture workers (%)',
        'Industry workers (%)',
        'Population 60+ (%)',
        'Population 40-59 (%)',
        'Physicians per 1000',
    ]
    
    # On identifie les colonnes à supprimer (celles > 60% NaN ET qui ne sont pas critiques)
    cols_to_drop = [
        col for col in missing_ratio[missing_ratio > 0.6].index 
        if col not in critical_columns
    ]
    
    print(f"Dropping {len(cols_to_drop)} columns with >60% missing values (excluding critical ones):")
    print(cols_to_drop)
    
    # Suppression effective
    df_merge = df_merge.drop(columns=cols_to_drop)
    # --------------------

    # Filter by Country Code
    df_filtered, dropped_countries = filter_high_nan(df_merge, "Country Code")
    print(f"Removing {len(dropped_countries)} countries with >60% missing values:")
    print(dropped_countries)
    
    # Drop duplicates
    df_filtered = df_filtered.drop_duplicates()

    print(f"Saving file {df_name} with {df_filtered.shape[0]} rows")
    df_filtered.to_csv(f"{BASE_PATH}Refined/{df_name}.csv", index=False)

### 6 Datasets (Data from 2010 to 2021)

In [39]:
df_COPD_prevalence_rate = df_COPD_prevalence_rate.astype({'Year': 'string'})
df_COPD_incidence_rate = df_COPD_incidence_rate.astype({'Year': 'string'})
df_asthma_prevalence_rate = df_asthma_prevalence_rate.astype({'Year': 'string'})
df_asthma_incidence_rate = df_asthma_incidence_rate.astype({'Year': 'string'})
df_tuberculosis_prevalence_rate = df_tuberculosis_prevalence_rate.astype({'Year': 'string'})
df_tuberculosis_incidence_rate = df_tuberculosis_incidence_rate.astype({'Year': 'string'})

df_health = [df_COPD_prevalence_rate, df_COPD_incidence_rate]

df_health_names = ["9019/COPD_prevalence_rate_s_3", "9019/COPD_incidence_rate_s_3"]

for i in range(len(df_health)):
    save_final_csv(df_health[i], df_filtered, df_health_names[i])

Dropping 5 columns with >60% missing values (excluding critical ones):
['Natural gas production', 'Sulfur emission', 'Nuclear power generation', 'Prevalence tobacco use', 'Total sales of agricultural pesticides (tonnes)']
Removing 0 countries with >60% missing values:
[]
Saving file 9019/COPD_prevalence_rate_s_3 with 5212 rows
Dropping 5 columns with >60% missing values (excluding critical ones):
['Natural gas production', 'Sulfur emission', 'Nuclear power generation', 'Prevalence tobacco use', 'Total sales of agricultural pesticides (tonnes)']
Removing 0 countries with >60% missing values:
[]
Saving file 9019/COPD_incidence_rate_s_3 with 5212 rows


We can use any of the final 6 datasets for our study (they combine different diseases, metrics, measures).